In [1]:
# Import modules
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup

In [2]:
import pickle

In [39]:
with open ('ids_pickle', 'rb') as fp:
    title_ids = pickle.load(fp)

In [40]:
# Grab gross, keywords, producers, music, cinematographer

In [41]:
# Url for box office info
url_boxoffice = 'http://www.imdb.com/title/{}/business?ref_=tt_dt_bus'

In [42]:
# Function to request and soup our search pages
def soupy(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    content = response.content
    soup = BeautifulSoup(content, "lxml")
    return soup

In [43]:
# Function to grab Box Office info

def get_gross(url):
    soup = soupy(url)
    Id = url.split('/')[4]
    try:
        Budget = soup.find(text=re.compile("Budget"))
        budget = Budget.next_element
        budget = budget.strip().replace(',','').replace('$','').split()[0]
    except AttributeError:
        budget = None
    try:
        Gross = soup.find(text=re.compile("Gross"))
        gross = Gross.next_element
        try:
            gross = gross.strip().replace(',','').replace('$','').split()[0]
        except:
            pass
    except AttributeError:
        gross = None
    try:    
        gross_w = soup.find(text=re.compile(r"[Ww]orldwide"))
        gross_w = gross_w.strip().replace(',','').replace('$','').split()[0]
    except AttributeError:
        gross_w = None
        
    gross_info.append([Id, budget, gross, gross_w]) 

In [44]:
# List to capture info
gross_info = []

In [109]:
# Loop over each id to grab box office for each title
for i in title_ids[16000:]:
    get_gross(url_boxoffice.format(i))

In [110]:
len(gross_info)

16136

In [199]:
# gross_info

In [111]:
gross_df = pd.DataFrame(gross_info)

In [112]:
gross_df.columns = ['imdbID', 'Budget', 'Gross', 'Gross_World']

In [113]:
# Drop any duplicates
gross_df.drop_duplicates(subset='imdbID', inplace=True)

In [114]:
# Drop any null ids that resulted from our connection being kicked
gross_df = gross_df[gross_df.imdbID.notnull()]

In [115]:
gross_df.reset_index(drop=True, inplace=True)

In [116]:
gross_df.shape

(16136, 4)

In [117]:
gross_df.head()

,imdbID,Budget,Gross,Gross_World
0,tt0437222,None,None,None
1,tt0050210,58000,None,None
2,tt0030725,None,None,None
3,tt0050212,3000000,5668000,None
4,tt0381681,2700000,5792822,None


In [118]:
gross_df.describe()

,imdbID,Budget,Gross,Gross_World
count,16136,6467,6698,2872
unique,16136,1240,5976,2707
top,tt0102138,20000000,ITL,60000000
freq,1,176,116,9


In [119]:
# Export dataframe to csv
gross_df.to_csv('gross.csv', encoding='utf-8')

In [120]:
gross_df = pd.read_csv('gross.csv', usecols=['imdbID', 'Budget', 'Gross', 'Gross_World'])

In [121]:
gross_df.head()

,imdbID,Budget,Gross,Gross_World
0,tt0437222,NaN,NaN,NaN
1,tt0050210,58000,NaN,NaN
2,tt0030725,NaN,NaN,NaN
3,tt0050212,3000000,5668000,NaN
4,tt0381681,2700000,5792822,NaN


In [122]:
gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16136 entries, 0 to 16135
Data columns (total 4 columns):
imdbID         16136 non-null object
Budget         6467 non-null object
Gross          6698 non-null object
Gross_World    2872 non-null object
dtypes: object(4)
memory usage: 504.3+ KB


In [123]:
gross_df.Gross = gross_df.Gross.apply(pd.to_numeric, errors='coerce')

In [124]:
gross_df.Budget = gross_df.Budget.apply(pd.to_numeric, errors='coerce')

In [125]:
gross_df.Gross_World = gross_df.Gross_World.apply(pd.to_numeric, errors='coerce')

In [126]:
gross_df.describe()

,Budget,Gross,Gross_World
count,6.069000e+03,6.273000e+03,2.856000e+03
mean,2.578905e+07,3.754193e+07,1.383029e+08
std,3.804192e+07,5.956840e+07,1.999365e+08
min,4.000000e+02,6.400000e+01,2.389000e+03
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,3.000000e+08,9.366274e+08,2.787965e+09


In [127]:
gross_cull = gross_df.dropna(axis=0, how='all', subset=['Budget', 'Gross', 'Gross_World'])

In [128]:
# Export dataframe with pickle
gross_cull.to_pickle('gross_cull.pkl')

In [129]:
gross_cull = pd.read_pickle('gross_cull.pkl')

In [130]:
gross_cull.describe()

,Budget,Gross,Gross_World
count,6.069000e+03,6.273000e+03,2.856000e+03
mean,2.578905e+07,3.754193e+07,1.383029e+08
std,3.804192e+07,5.956840e+07,1.999365e+08
min,4.000000e+02,6.400000e+01,2.389000e+03
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,3.000000e+08,9.366274e+08,2.787965e+09


In [131]:
gross_cull.head()

,imdbID,Budget,Gross,Gross_World
1,tt0050210,58000.0,NaN,NaN
3,tt0050212,3000000.0,5668000.0,NaN
4,tt0381681,2700000.0,5792822.0,NaN
5,tt0472160,15000000.0,10011274.0,NaN
6,tt0209475,35000000.0,60400856.0,94728529.0


In [132]:
# Replcae Gross with Gross world if gross world exists
cond = gross_cull.Gross_World > 0

gross_cull.Gross[cond] = gross_cull.Gross_World[cond]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [133]:
# Calculate Box Office performance
gross_cull['performance'] = (gross_cull['Gross'] - gross_cull['Budget'])/gross_cull['Budget']

In [134]:
gross_cull.head(10)

,imdbID,Budget,Gross,Gross_World,performance
1,tt0050210,58000.0,NaN,NaN,NaN
3,tt0050212,3000000.0,5668000.0,NaN,0.889333
4,tt0381681,2700000.0,5792822.0,NaN,1.145490
5,tt0472160,15000000.0,10011274.0,NaN,-0.332582
6,tt0209475,35000000.0,94728529.0,94728529.0,1.706529
8,tt0871426,30000000.0,60269340.0,NaN,1.008978
16,tt0103639,28000000.0,504050219.0,504050219.0,17.001794
17,tt0098384,NaN,95904091.0,95904091.0,NaN
18,tt1185416,55000000.0,32669555.0,NaN,-0.406008
19,tt0208874,20000000.0,22361811.0,22361811.0,0.118091


In [137]:
gross_cull.drop('Gross_World', axis=1, inplace=True)

In [138]:
gross_cull.head()

,imdbID,Budget,Gross,performance
1,tt0050210,58000.0,NaN,NaN
3,tt0050212,3000000.0,5668000.0,0.889333
4,tt0381681,2700000.0,5792822.0,1.145490
5,tt0472160,15000000.0,10011274.0,-0.332582
6,tt0209475,35000000.0,94728529.0,1.706529


In [139]:
# Export dataframe with pickle
gross_cull.to_pickle('gross_perf.pkl')

In [140]:
gross_cull.describe()

,Budget,Gross,performance
count,6.069000e+03,6.433000e+03,4801.000000
mean,2.578905e+07,6.938995e+07,7.686365
std,3.804192e+07,1.473397e+08,189.526776
min,4.000000e+02,6.400000e+01,-0.999510
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,3.000000e+08,2.787965e+09,12889.386667


---
### Grab producer credits

In [113]:
# Url for grabbibg ore credit info
url_credits = 'http://www.imdb.com/title/{}/fullcredits?ref_=tt_ov_st_sm'

In [117]:
# Function to grab producer info
def get_producers(url):
    soup = soupy(url)
    Id = url.split('/')[4]
    try:
        produced = soup.find(text=re.compile("Produced by"))
        nexty = produced.findNext('table')  
        names = nexty.findAll(class_='name')
        producers = []
        for i in names:
            producers.append(i.text.strip())
    except AttributeError:
        producers = None
        
    producer_info.append([Id, producers])

In [118]:
# List to capture producer info
producer_info = []

In [161]:
for i in title_ids[16000:]:
    get_producers(url_credits.format(i))

In [170]:
producer_df = pd.DataFrame(producer_info)

In [171]:
producer_df.columns = ['imdbID', 'Producers']

In [172]:
# Drop any duplicates
producer_df.drop_duplicates(subset='imdbID', inplace=True)

In [173]:
# Drop any null ids that resulted from our connection being kicked
producer_df = producer_df[producer_df.imdbID.notnull()]

In [174]:
producer_df.shape

(16137, 2)

In [175]:
producer_df.head()

,imdbID,Producers
0,tt0437222,None
1,tt0050210,"[Jacques R. Marquette, Dale Tate, Joy N. Houck..."
2,tt0030725,"[George M. Merrick, Louis Weiss]"
3,tt0050212,[Sam Spiegel]
4,tt0381681,"[Isabelle Coulet, John Sloss, Anne Walker-McBay]"


In [176]:
# Export dataframe with pickle
producer_df.to_pickle('producer_df.pkl')

In [141]:
producer_df.head()

NameError: name 'producer_df' is not defined